In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import warnings
warnings.filterwarnings(action='ignore')

from matplotlib import pyplot as plt
from matplotlib import rcParams
%matplotlib inline

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pathlib import Path
from collections import  Counter

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [2]:
# 파일 불러오기
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test_x.csv')
sample_submission = pd.read_csv('data/sample_submission.csv', encoding = 'utf-8')

In [3]:
# 클렌징: 부호 제거, 소문자 변환
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text.lower())

train['text']=train['text'].apply(alpha_num)

In [4]:
# 스톱워즈 제거 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 스톱워즈
stopwords = ['the', 'and', 'i', 'to', 'of', 'a', 'in', 'to', 'had', 'he', 'man', 'have', 'are',
             'that', 'you', 'was', 'with', 'form', 'his', 'as', 'odin', 'said', 'one']

In [5]:
# test 세트도 소문자로 변환
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)

test['text'] = test['text'].str.lower()
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [6]:
# train - test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [7]:
X_train

array(['almost choking there so much so much wanted say but strange exclamations were all came from lips pole gazed fixedly at him at bundle notes hand looked at evident perplexity',
       'your sister asked for it suppose',
       'she engaged day she walked perusing janes last letter dwelling on some passages which proved jane not written spirits when instead being again surprised by mr she saw on looking up meeting her putting away letter immediately forcing smile she',
       ..., 'your sincere wellwisher friend sister lucy',
       'then wanted me lend money',
       'it certainly not occurred me before but yes should like'],
      dtype='<U1855')

In [20]:
trn = pd.read_csv('data/train.csv')

In [22]:
tst = pd.read_csv('data/train.csv')

In [26]:
tst = pd.read_csv('data/train.csv')

In [27]:
trn

,index,text,author
0,0,he was almost choking there was so much so muc...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in peru...,1
3,3,the captain was in the porch keeping himself c...,4
4,4,have mercy gentlemen odin flung up his hands d...,3
...,...,...,...
54874,54874,is that you mr smith odin whispered i hardly d...,2
54875,54875,i told my plan to the captain and between us w...,4
54876,54876,your sincere wellwisher friend and sister luc...,1
54877,54877,then you wanted me to lend you money,3


In [28]:
# 스톱워즈 제거 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 스톱워즈
stopwords = ['the', 'and', 'i', 'to', 'of', 'a', 'in', 'to', 'had', 'he', 'man', 'have', 'are',
             'that', 'you', 'was', 'with', 'form', 'his', 'as', 'odin', 'said', 'one']

In [29]:
trn

,index,text,author
0,0,he was almost choking there was so much so muc...,3
1,1,your sister asked for it i suppose,2
2,2,she was engaged one day as she walked in peru...,1
3,3,the captain was in the porch keeping himself c...,4
4,4,have mercy gentlemen odin flung up his hands d...,3
...,...,...,...
54874,54874,is that you mr smith odin whispered i hardly d...,2
54875,54875,i told my plan to the captain and between us w...,4
54876,54876,your sincere wellwisher friend and sister luc...,1
54877,54877,then you wanted me to lend you money,3


In [32]:
# test 세트도 소문자로 변환
trn['text'] = trn['text'].apply(alpha_num).apply(remove_stopwords)

tst['text'] = tst['text'].str.lower()
tst['text'] = tst['text'].apply(alpha_num).apply(remove_stopwords)

In [48]:
# train - test 분리
X_trn = np.array([x for x in train['text']])
X_tst = np.array([x for x in test['text']])
y_trn = np.array([x for x in train['author']])

In [49]:
X_train

array(['almost choking there so much so much wanted say but strange exclamations were all came from lips pole gazed fixedly at him at bundle notes hand looked at evident perplexity',
       'your sister asked for it suppose',
       'she engaged day she walked perusing janes last letter dwelling on some passages which proved jane not written spirits when instead being again surprised by mr she saw on looking up meeting her putting away letter immediately forcing smile she',
       ..., 'your sincere wellwisher friend sister lucy',
       'then wanted me lend money',
       'it certainly not occurred me before but yes should like'],
      dtype='<U1855')

# 머신러닝으로!! Bag-of-Words 피처

# NLTK

In [35]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer

In [50]:
s = trn.text[4]
print(s)

mercy gentlemen flung up hands dont write anyway some shame here ive torn my heart asunder before seize opportunity fingering wounds both halves oh my god


In [51]:
tokens = word_tokenize(s)
print(tokens)

['mercy', 'gentlemen', 'flung', 'up', 'hands', 'dont', 'write', 'anyway', 'some', 'shame', 'here', 'ive', 'torn', 'my', 'heart', 'asunder', 'before', 'seize', 'opportunity', 'fingering', 'wounds', 'both', 'halves', 'oh', 'my', 'god']


In [52]:
lemmatizer = WordNetLemmatizer()
[lemmatizer.lemmatize(t) for t in tokens]

['mercy',
 'gentleman',
 'flung',
 'up',
 'hand',
 'dont',
 'write',
 'anyway',
 'some',
 'shame',
 'here',
 'ive',
 'torn',
 'my',
 'heart',
 'asunder',
 'before',
 'seize',
 'opportunity',
 'fingering',
 'wound',
 'both',
 'half',
 'oh',
 'my',
 'god']

In [53]:
stemmer = SnowballStemmer("english")
[stemmer.stem(t) for t in tokens]

['merci',
 'gentlemen',
 'flung',
 'up',
 'hand',
 'dont',
 'write',
 'anyway',
 'some',
 'shame',
 'here',
 'ive',
 'torn',
 'my',
 'heart',
 'asund',
 'befor',
 'seiz',
 'opportun',
 'finger',
 'wound',
 'both',
 'halv',
 'oh',
 'my',
 'god']

In [61]:
vec = CountVectorizer(alpha_num, tokenizer=word_tokenize, stop_words=['the', 'and', 'i', 'to', 'of', 'a', 'in', 'to', 'had', 'he', 'man', 'have', 'are',
             'that', 'you', 'was', 'with', 'form', 'his', 'as', 'odin', 'said', 'one'], ngram_range=(1, 2), min_df=100)
X_cnt = vec.fit_transform(trn['text'])
print(X_cnt.shape)

(54879, 2426)


In [62]:
X_cnt[0, :50].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [63]:
vec = TfidfVectorizer(alpha_num, tokenizer=word_tokenize, stop_words=['the', 'and', 'i', 'to', 'of', 'a', 'in', 'to', 'had', 'he', 'man', 'have', 'are',
             'that', 'you', 'was', 'with', 'form', 'his', 'as', 'odin', 'said', 'one'], ngram_range=(1, 3), min_df=50)
X = vec.fit_transform(trn['text'])
X_tst = vec.transform(tst['text'])
print(X.shape, X_tst.shape)

(54879, 5121) (54879, 5121)


In [64]:
X[0, :50].todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])

# 로지스틱회귀모델

In [65]:
target_col = 'author'
n_fold = 5
n_class = 5
seed = 42

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [67]:
y = trn.author.values
y.shape

(54879,)

In [68]:
p = np.zeros((X.shape[0], n_class))
p_tst = np.zeros((X_tst.shape[0], n_class))
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
    clf = LogisticRegression()
    clf.fit(X[i_trn], y[i_trn])
    p[i_val, :] = clf.predict_proba(X[i_val])
    p_tst += clf.predict_proba(X_tst) / n_class

In [69]:
print(f'Accuracy (CV): {accuracy_score(y, np.argmax(p, axis=1)) * 100:8.4f}%')
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p):8.4f}')

Accuracy (CV):  69.4710%
Log Loss (CV):   0.8421
